In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pathlib

In [22]:
def augment_data(image):
    image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
    image = tf.image.resize_with_crop_or_pad(image, 180, 180) # Add 6 pixels of padding
    image = tf.image.random_crop(image, size=[150, 150, 3]) # Random crop back to 28x28
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness

    return image

In [3]:
directory = "D:/dataset/flower_photos"

In [4]:
train_dir = pathlib.Path(directory)

In [5]:
CLASS_NAMES = np.array([item.name for item in train_dir.glob('*') if item.name != "LICENSE.txt"])

In [6]:
CLASS_NAMES

array(['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips'],
      dtype='<U10')

In [7]:
full_dataset = tf.data.Dataset.list_files(str(train_dir/'*/*'))

In [8]:
validation_split = 0.2

In [9]:
DATASET_SIZE = len(list(full_dataset))
print("Dataset size: ", DATASET_SIZE)
train_size = int((1-validation_split) * DATASET_SIZE)
print("train size: ", train_size)
train_dataset = full_dataset.take(train_size)
validation_dataset = full_dataset.skip(train_size)

Dataset size:  3670
train size:  2936


In [10]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] == CLASS_NAMES

In [12]:
get_label("D:\\dataset\\flower_photos\\daisy\\5547758_eea9edfd54_n.jpg").numpy()

array([ True, False, False, False, False])

In [23]:
def load_img(image_path):
    img = tf.io.read_file(image_path)
    
    # https://stackoverflow.com/questions/44942729/tensorflowvalueerror-images-contains-no-shape
    img = tf.image.decode_image(img, 3, expand_animations=False)
    
    img = tf.cast(img, tf.float32)
    return img

In [24]:
# normalizing the images to [-1, 1]
def normalize(image):
    image = (image / 127.5) - 1
    return image

In [30]:
def resize(image,height, width):
    image = tf.image.resize(image, (height, width),
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    #image = tf.image.resize_with_crop_or_pad(image, height, width)
    return image

In [25]:
def load_image_with_label(image_path):
    label = get_label(image_path)
    img = load_img(image_path)
    return img, label

In [26]:
def load_image_train(image_file):
    image, label = load_image_with_label(image_file)
    image = augment_data(image)
    image = normalize(image)
    
    return image, label

In [27]:
def load_image_test(image_file):
    image, label = load_image_with_label(image_file)
    image = resize(image, 150, 150)
    image = normalize(image)

    return image, label

In [18]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [28]:
train_dataset = train_dataset.map(load_image_train)
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

In [31]:
validation_dataset = validation_dataset.map(load_image_test)
validation_dataset = validation_dataset.batch(BATCH_SIZE)

In [32]:
base_model = tf.keras.applications.VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [33]:
base_model.trainable = False

In [37]:
n_class =len(CLASS_NAMES)

In [40]:
flatten_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(100, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.5)
prediction_layer = tf.keras.layers.Dense(n_class, activation='softmax')

In [41]:
model = tf.keras.Sequential([
  base_model,
  flatten_layer,
  dense_layer,
  dropout_layer,
  prediction_layer
])

In [ ]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from train_utils import get_scheduler
from datetime import datetime
from keras.callbacks import TensorBoard

In [ ]:
scheduler = get_scheduler(model, "lr.txt")
lr_decay = LearningRateScheduler(scheduler, verbose=1)

In [ ]:
subdir = datetime.strftime(datetime.now(), '%Y%m%d-%H%M%S')
model_dir = os.path.join(os.path.expanduser("/mnt/datasets/checkpoints/logo/"), subdir)
if not os.path.isdir(model_dir):  # Create the model directory if it doesn't exist
    os.makedirs(model_dir)

    
filepath = os.path.join(model_dir, "weights-{epoch:02d}.hdf5")

checkpoint = ModelCheckpoint(filepath, verbose=0, save_best_only=False, save_weights_only=True, mode='auto', period=1)

In [ ]:
log_dir = os.path.join(os.path.expanduser("./logs/vggg16"), subdir)
if not os.path.isdir(log_dir):  # Create the log directory if it doesn't exist
    os.makedirs(log_dir)
    
tensorboard=TensorBoard(log_dir=log_dir)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset,
                    epochs=100,
                    validation_data=validation_dataset,
                    callbacks=[lr_decay, checkpoint, tensorboard])